# edit anywhere

Purpose: change the web however you want, save and restore it from github, treat the web like one giant web of gists.




## introduction

First attempt at this app was using megamind.bot app, the editor takes an input URL, and a filename.


 It loads a website from crawled data from Selenium/data collection.ipynb. It then sent these parameters to the client app for displaying in the same frame.

The new implementation should act more like a marketing website. Enter a URL in a box in the middle like Google, use the controls that appear on the copied page content. Use different methods to get the page content such as Selenium crawler or simple phantom browser.

After the content is crawled, make any changes that save to a single Gist for the entire domain. Load the gist from domain when the page is loaded (plugin phase 2).

TODO:

Fix SPA/PWA apps and canvas copies.

Move page processing to generalized Selenium, convert scripts.


## gist

Read and write files from gist.



### read gist files


#### the code

read gist files?


In [ ]:
var GitHubApi = require('github');

// commit changes to github
function getGist(gist) {
    const github = new GitHubApi({
        host: 'api.github.com'
    });
    /*
    github.authenticate({
        type: 'basic',
        username: process.env.USERNAME,
        password: process.env.PASSWORD
    });
    */

    return github.gists.get({id: gist})
        .then(r => r.data.files);
}


### write gist files

write gist files?


#### the code 



## ckeditor client

Use CKEditor and some scripts to apply some ACLs to the page and output.



### apply ACL


#### the code
apply acl to html?


In [ ]:
// scan using an acl list, similar to easylist?
// TODO: accept formats:
//    {"selector": "selector"}
//    {"glob-url@selector": "glob-template-path@selector"}
//    {"selector": "html-file@selector"}
//    {"selector": "html-file@xpath"} ?
//    {"glob-file": {"glob-url"...} || ["selector"]}
const paths = JSON.parse('[]');

### load ckeditor


#### the code


In [ ]:
var ASTQ = require('astq');
var ASTY = require('asty');
var glob = require('glob');
var importer = require('../Core');
var loadScraped = importer.import('get scraped page');

// git 
function gitEditor(gist, file) {
    // TODO: use a Github repo as the input
    const HOST_ID = 'webtest123'
    gist = gist || 'act.com' || 'a572d0830ae72b962e12a57adaec7c52';
    file = file || 'index';
    return loadScraped(gist, file)
        .then(files => typeof files[ file + '-acl.json' ] !== 'undefined'
            ? Promise.resolve(files)
            : getGist(gist))
        .then(files => {
            
        })
}
module.exports = gitEditor;

if(typeof $$ !== 'undefined') {
    $$.async();
    gitEditor()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


## tools


### restrain CSS

Replace all CSS rules with a container ID to restain it's affects on the page.


#### the code

restrain css? 

scope css?



In [ ]:
var css = require('css');

function prefixCssRules(str, prefix) {
    const ast = css.parse(str);
    // TODO: add a check for media queries
    ast.stylesheet.rules.forEach(r => {
        if(typeof r.selectors === 'undefined') {
            return;
        }
        r.selectors.forEach((s, i) => {
            r.selectors[i] = prefix + ' ' + s;
        });
    })
    return css.stringify(ast);
}

module.exports = prefixCssRules;


### TODO: express crawl middleware

Serve every static address from a cache crawl json file.

TODO: move this to data collection tools


### read crawl files

Load matching files from a crawled cache json file. See Selenium/data collection.ipynb for more information on crawl cache json.

TODO: move this to data collection.ipynb tools


#### the code 

read crawl files?


In [4]:
var path = require('path')
var fs = require('fs')
var {URL} = require('url')
var uuid = require('uuid/v1')
var importer = require('../Core')
var {glob} = importer.import('glob files')
var {minimatch} = importer.import('minimatch')
var {selectDom} = importer.import('select tree')
var prefixCssRules = importer.import('scope css')

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(PROFILE_PATH, 'Collections/crawls');

function getAllLinks(doc) {
    return selectDom({
        links: '//a/@href|//*[@src]/@src',
        styles: '//link/@href|style/@text()'
    }, doc)
}

function linkToAbsolute(base, link) {
    return link.trim().match(/^\/\//ig)
       ? ('https:' + link.trim())
       : (link.trim().match(/^https?:\/\//ig)
         ? link.trim()
         : base.replace(/\/$/ig, '') + '/' + link.trim().replace(/^\//ig, ''))
}

function findCache(host) {
    const crawl = glob(
        '**/*' + host.replace(/[^a-z0-9_-]/ig, '_') + '*',
        project
    )
    crawl.sort((a, b) => {
        return fs.statSync(b).mtime.getTime() - fs.statSync(a).mtime.getTime()
    })
    return crawl
}

function matchPage(match, search, hostname) {
    return search.includes(match)
        || minimatch(search, match)
        || !match
        && search.match(/https?:\/\/[^\/]*\/?$/ig)
        && search.includes(hostname)
}

function loadScraped(url) {
    if(typeof url == 'string') {
        url = new URL(url);
    }
    var host = url.hostname
    var file = url.filename
    console.log(file)
    var cache = findCache(host)
    
    if(!cache[0]) {
        return
    }
    const crawl = JSON.parse(fs.readFileSync(cache[0]).toString());
    const entry = crawl.filter(r => matchPage(file, r.url, host))[0];
    
    if(entry) {
        const styles = entry.styles.map(s => linkToAbsolute(entry.url, s));
        console.log(styles)
        const stylesheets = crawl
            // TODO: fix this body doesn't work, remove <body tags from downloaded page
            .filter(r => styles.includes(r.url))
            .map(r => selectDom(['body > pre'], r.content)[0].textContent)
            .filter(r => r && !r.match(/<[^>]*?>.*?<\/[^>]*?>/ig) && !r.match(/<style/ig))
            .join('\n');
        const HOST_ID = uuid().substr(0, 5) + '-' + host.replace(/[^a-z0-9_-]/ig, '_')
 
        const css = prefixCssRules(stylesheets, '#' + HOST_ID);
        return `<style scoped id="${HOST_ID}">${css}</style>${entry.content}`;
    }
}

module.exports = loadScraped

//var importer = require('../Core')
//var loadScraped = importer.import('read crawl files')

if(typeof $$ != 'undefined') {
    var scraped = loadScraped('https://act.com')
    $$.html(scraped)
}

importing glob files - 1 cell - files.ipynb[2]
importing minimatch - 1 cell - minimatch.ipynb[2]
importing select tree - 1 cell - patterns.ipynb[2]
importing scope css - 1 cell - ckeditor git acl.ipynb[5]
compiling /Users/briancullinan/jupyter_ops/Frontends/ckeditor git acl.ipynb[5] aka scope css with module,importer,exports,__dirname,__filename,$$
undefined
[
  'https://fonts.googleapis.com/css?family=Poppins:300,400,600,700',
  'https://www.act.com',
  'https://www.act.com/Sitefinity/WebsiteTemplates/act2017/App_Theme/act2017/global/grid.css?v=636361570495580656',
  'https://www.act.com/Sitefinity/WebsiteTemplates/act2017/App_Theme/act2017/global/bootstrap.css?v=636402989902731538',
  'https://www.act.com/Sitefinity/WebsiteTemplates/act2017/App_Theme/act2017/global/main.css?v=636505085974950034',
  'https://www.act.com/Sitefinity/WebsiteTemplates/act2017/App_Theme/act2017/global/slippry.css?v=636334759299377029',
  'https://www.act.com/Sitefinity/WebsiteTemplates/act2017/App_Theme/ac

<!-- Google Tag Manager -->
 <iframe src="//www.googletagmanager.com/ns.html?id=GTM-KW52GZ"
 height="0" width="0" style="display:none;visibility:hidden"></iframe> 
 
 <!-- End Google Tag Manager -->

 <!--[if lt IE 9]>
 You are using an outdated browser. Please upgrade your browser to improve your experience. 
 <![endif]-->
 
 
 
 
 
 




 

 
 
 
 <!-- ============================================================================== -->
 

 
 
 
 
 
 
 
 
 
 
 
 
 Select a region
 
 English (UK) 
 English (ANZ) 
 Deutsch (Deutschland) 
 Français (France) 
 Français (Canada) 
 Nederlands (Nederland) 
 Polski (Polska) 
 
 
 
 
 866-873-2006 
 
 
 FREE Trial
 
 
 
 
 
 <!-- END MOBILE NAV -->
 
 <!-- -->
 
 
 
 
 
 
 
 
 
 
 
 
 Select a region
 
 English (UK) 
 English (ANZ) 
 Deutsch (Deutschland) 
 Français (France) 
 Français (Canada) 
 Nederlands (Nederland) 
 Polski (Polska) 
 
 
 
 
 866-873-2006 
 
 
 
 FREE Trial
 
 
 
 
 
 
 
 
 
 
 
 
 Why Act! 
 
 Why choose Act! CRM 
 Customer Testimonials 
 
 
 
 Products & Pricing 
 
 Act! Products & Pricing 
 Act! Features & Benefits 
 Act! emarketing 
 Act! Connect 
 Act! Companion Mobile App 
 Handheld Contact Mobile Add-on 
 What's New 
 
 
 Customer Resources 
 
 Act! Customer Resources 
 Act! emarketing Resources 
 Act! Essentials Resources 
 Getting Started with Act! 
 Act! Blog 
 
 
 
 Global Partner Network 
 Contact Us 
 
 
 
 
 
 
 
 
 
 

 <!-- ============================================================================== -->
 
 


 
 
 
 
 
 
 
 
 
 
 The new Ask Act!™ with Amazon Alexa lets you interact with Act! through verbal requests!


 
 
 
 
 Learn more 


 
 
 
 
 
 
 

 
 
 
 
 
 
 Flexible CRM that’s uniquely yours 
 Act! makes it easy to build lasting relationships and provides the freedom to tailor an Act! experience to your business and industry needs - your adaptable, everywhere, connected workspace. 


 
 
 
 
 Get started for free 
 
 No credit card required. 
 
 Request a demo 
 
 
 


 
 
 866-873-2006 


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 Adaptable 


 
 
 


 Act! CRM works like you work, equipping you with a variety of configuration options from lightly personalized to completely customized.


 
 
 
 
 Everywhere 
 


 
 
 


 Act! CRM goes where you go, enabling you to be productive from wherever you work—the office, home, or the road—whether you’re online or offline.


 
 
 
 
 Connected 


 
 
 


 Act! CRM likes what you like, keeping you connected to popular business apps like Outlook®, Google™, QuickBooks® Online, and more.


 
 
 
 
 
 Why choose 


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Build lasting relationships 
 Make every interaction count with quick, organized access to highly personalized customer details. 
Keep rich contact details, emails, notes, history, activities, social media updates, and more in one easy to find place—the Act! contact record. 
 
 Explore features 
 
   


 
 
 
 
 
 
 


 
 
 
 
 
 
 Fuel business growth 
 Drive demand and close deals faster with integrated email marketing and powerful sales tools. 
 Create and send professional, mobile-friendly campaigns proven to drive sales to the contacts and groups you keep in Act!. Manage leads through to close using an out-of-the-box process or one you customize. 
 Explore features      
   


 
 
 
 
 
 
 


 
 
 
 
 
 
 Make informed decisions 
 Gain a deep understanding of business performance with rich, actionable insights. 
 
See dynamic, visual snapshots of real-time metrics with interactive, graphical dashboards. Dig deeper with 50+ preconfigured reports or create custom reports to fit your unique needs. 
 Explore features 
   
 
   


 
 
 
 
 
 
 


 
 
 
 
 
 
 
 Next Prev 1 2 3 
 
 
 
 
 
 
 
 
 
 
 What you get with Act! CRM 
 
 Powerful sales and marketing features that fuel business growth. 
 Freedom to tailor an Act! experience to fit your unique needs. 
 Flexible deployment with Cloud, on-premises, and hybrid options. 
 Exclusive Act!

#### test crawl cache loader

